In [11]:
# 绘制均值roc代码
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

def average_results(root_dir, sub_dir, name, out_name):
    
    dir = os.path.join(root_dir, sub_dir)
    df = pd.DataFrame()
    for file_name in os.listdir(dir):
        if name in file_name:
            df = pd.read_csv(os.path.join(dir, file_name))

    df_mean = []
    for col in df:
        if col == 'model':
            df = df.drop('model', axis=1)
        elif col == 'cm':
            df_mean.append(np.array([eval(i) for i in df[col].tolist()]).mean(axis=0).astype(int).tolist()) 
        else:
            df_mean.append(str(round(df[col].mean(), 4)) + '±' + str(round(df[col].std(), 4)))
            
    df.insert(0, 'Run', [f'{i+1}_th' for i in range(len(df))], allow_duplicates=False)
    df = df.round(4)
    df.loc[len(df)] = ['Mean'] + df_mean
    df.to_csv(os.path.join(dir, out_name), index=False)
    

def get_rocs(root_dir, sub_dir):
    
    fprs, tprs = [], []
    roc_dir = os.path.join(root_dir, sub_dir)
    for roc_file in os.listdir(roc_dir):
        if 'roc' in roc_file:
            roc_path = os.path.join(roc_dir, roc_file)
            roc_df = pd.read_csv(roc_path)
            fprs.append(roc_df['fpr'])
            tprs.append(roc_df['tpr'])
        elif 'ave_test_mean' in roc_file:
            avg_auc = pd.read_csv(os.path.join(roc_dir, roc_file))['auc'].iloc[-1].split('±')
            avg_auc = list(map(float,avg_auc))
    
    return fprs, tprs, avg_auc


def draw_roc(fprs, tprs, mean_std_auc=None):

    colors = ['tomato','chocolate','orange','gold','palegreen','turquoise','cyan','deepskyblue','mediumslateblue','hotpink']
    roc_fig = plt.figure()
    fig = plt.subplot(1, 1, 1)
    fig.set_title('AMPpred_MFA ROC curve on XUAMP', fontsize=14)
    fig.set_xlim([0.0, 1.0])
    fig.set_ylim([0.0, 1.0])
    fig.set_xlabel('False Positive Rate', fontsize=12)
    fig.set_ylabel('True Positive Rate', fontsize=12)
    fig.plot([0, 1], [0, 1], linestyle='--', lw=0.5, color='r', label='Random', alpha=0.8)

    aucs = []
    mean_fpr = np.linspace(0, 1, max([len(i) for i in fprs]))
    interp_tprs = []

    for i in range(len(fprs)):
        fig.plot(fprs[i], tprs[i], lw=1, alpha=0.8, color=colors[i % len(colors)],
                 label='Exp. {} (AUC = {:.3f})'.format(i + 1, metrics.auc(fprs[i], tprs[i])))
        interp_tpr = np.interp(mean_fpr, fprs[i], tprs[i])
        interp_tprs.append(interp_tpr)
        aucs.append(metrics.auc(fprs[i], tprs[i]))

    mean_tpr = np.mean(interp_tprs, axis=0)
    mean_tpr[0], mean_tpr[-1] = 0, 1
    if not mean_std_auc:
        mean_auc = metrics.auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
    else:
        mean_auc = mean_std_auc[0]
        std_auc = mean_std_auc[1]

    fig.plot(mean_fpr, mean_tpr, color='dimgrey', label=r'Mean (AUC = {:.3f} ± {:.3f})'.format(mean_auc, std_auc),
             lw=1.5, alpha=0.6)

    std_tpr = np.std(interp_tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    # fig.fill_between(mean_fpr, tprs_lower, tprs_upper, color='slategray', alpha=0.2, label=r'± 1 std. dev.')

    fig.legend(loc="lower right", fontsize=10)

    return roc_fig

root_dir = 'trainstat/cls'
sub_dir = '0.002'
    
average_results(root_dir, sub_dir, 'ave_test_stat', 'ave_test_mean.csv')

fig = draw_roc(*get_rocs(root_dir, sub_dir))
fig.savefig(os.path.join(root_dir, sub_dir, 'ave_curve.png'), dpi=300)
plt.close()